In [1]:
library(igraph)
library(rjson)
library(data.table)


Attaching package: 'igraph'

The following objects are masked from 'package:stats':

    decompose, spectrum

The following object is masked from 'package:base':

    union



In [22]:
#Question6
json_sfc<-fromJSON(file='data/san_francisco_censustracts.json')
attributes(json_sfc)
csv_sfc<-read.csv(file='data/san_francisco-censustracts-2017-4-All-MonthlyAggregate.csv',header=TRUE,sep=",")
setDT(csv_sfc)
csv_sfc<-csv_sfc[month==12][, c("sourceid", "dstid", "mean_travel_time")]

$names
[1] "type"     "features"

In [23]:
temps<-json_sfc $ features
l<-length(temps)
print(l)
display_temp<-c()
x_temp<-c()
y_temp<-c()
movement_id_temp<-c()
location_temp<-list()
for(i in 1:l){
    temp=temps[[i]]
    display_temp<-c(display_temp,temp$properties$DISPLAY_NAME)
    location<-temp$geometry$coordinates
    location2<-split(unlist(location),1:2)
    x_ave=mean(location2[[1]])
    y_ave=mean(location2[[2]])
    x_temp<-c(x_temp,x_ave)
    y_temp<-c(y_temp,y_ave)
    location3<-c()
    location3<-c(location3,x_ave)
    location3<-c(location3,y_ave)
    location_temp[[i]]<-location3
    movement_id_temp<-c(movement_id_temp,temp$properties$MOVEMENT_ID)
}
g<-graph.data.frame(d=csv_sfc,directed=FALSE)
print(g)
print(vcount(g))
print(ecount(g))
display<-as.list(setNames(display_temp,movement_id_temp))
xtemp<-as.list(setNames(x_temp,movement_id_temp))
ytemp<-as.list(setNames(y_temp,movement_id_temp))
location4<-as.list(setNames(location_temp,movement_id_temp))
V(g)$DISPLAY_NAME<-display[V(g)$name]
V(g)$x1<-xtemp[V(g)$name]
V(g)$y1<-ytemp[V(g)$name]
V(g)$LOCATION<-location4[V(g)$name]
g<-simplify(g, remove.multiple=TRUE,edge.attr.comb="mean")
temp1<-components(g)
gcc<-induced.subgraph(g, which(temp1$membership==which.max(temp1$csize)))
print(gcc)
print(vcount(gcc))
print(ecount(gcc))

[1] 2710
IGRAPH d529052 UN-- 1887 538998 -- 
+ attr: name (v/c), mean_travel_time (e/n)
+ edges from d529052 (vertex names):
 [1] 1938--1952 1747--629  1872--1919 1871--1929 81  --2319 1873--1909
 [7] 232 --809  1891--1729 2423--62   650 --1829 680 --1529 542 --2251
[13] 604 --1631 660 --1729 1329--1917 2421--82   1736--739  2021--1192
[19] 550 --2171 1721--1308 1700--1518 176 --2145 1721--889  536 --2311
[25] 74  --2466 288 --326  169 --2215 99  --2216 1755--549  646 --1211
[31] 2315--528  2322--458  2307--608  1390--111  641 --1261 1872--1946
[37] 2357--108  1454--64   1729--809  2355--128  722 --1207 2307--612 
[43] 98  --2226 1074--464  1058--624  614 --1531 151 --2395 2341--268 
+ ... omitted several edges
[1] 1887
[1] 538998
IGRAPH d5a24de UN-- 1880 311802 -- 
+ attr: name (v/c), DISPLAY_NAME (v/x), x1 (v/x), y1 (v/x), LOCATION
| (v/x), mean_travel_time (e/n)
+ edges from d5a24de (vertex names):
 [1] 1938--1747 1938--1872 1938--1873 1938--232  1938--1891 1938--550 
 [7] 1938--172

In [11]:
a<-0
b<-0
for(i in 1: vcount(g)){
    for(j in i:vcount(g)){
        c<-sqrt((V(g)[i]$x-V(g)[j]$x)^2+(V(g)[i]$y-V(g)[j]$y)^2)
        id<-get.edge.ids(g,c(i,j),directed=F,error=F)
        print(id)
        if(id!=0){
            a<-a+c
            b<-b+E(g)[id]$mean_travel_time
            print("show")
            print(a)
            print(b)
        }
    }
}
mean_travel_time_ratio<-b/a
print(mean_travel_time_ratio)

ERROR: Error in V(g)[i]$x - V(g)[j]$x: non-numeric argument to binary operator


In [24]:
#question7
mstree<-mst(gcc,weights=E(gcc)$mean_travel_time)
mstree

IGRAPH d9015b2 UN-- 1880 1879 -- 
+ attr: name (v/c), DISPLAY_NAME (v/x), x1 (v/x), y1 (v/x), LOCATION
| (v/x), mean_travel_time (e/n)
+ edges from d9015b2 (vertex names):
 [1] 1938--1241 1938--700  1938--2631 1747--1455 1747--528  1747--1746
 [7] 1872--563  1871--563  1871--1381 1871--390  81  --1701 81  --1295
[13] 81  --1772 1873--2488 232 --2155 232 --2513 1891--1868 2423--169 
[19] 2423--387  2423--602  2423--1086 650 --1703 650 --206  680 --1748
[25] 680 --1461 542 --2700 542 --2503 542 --543  604 --1595 604 --1102
[31] 660 --1296 660 --1776 660 --967  1329--1311 1329--824  2421--2225
[37] 2421--2214 1736--2251 1736--1737 2021--1180 2021--2066 550 --702 
+ ... omitted several edges

In [21]:
street_address<-list()
for(i in 1:5){
    edge<-E(mstree)[i]
    print(edge)
    temp<-ends(mstree,edge)
    street_address[[i]]<-c(V(mstree)[temp[1]]$DISPLAY_NAME,V(mstree)[temp[2]]$DISPLAY_NAME,edge$mean_travel_time)
}
print(street_address)

+ 1/1879 edge from 7ab17d6 (vertex names):
[1] 1938--1241
+ 1/1879 edge from 7ab17d6 (vertex names):
[1] 1938--700
+ 1/1879 edge from 7ab17d6 (vertex names):
[1] 1938--2631
+ 1/1879 edge from 7ab17d6 (vertex names):
[1] 1747--1455
+ 1/1879 edge from 7ab17d6 (vertex names):
[1] 1747--528
[[1]]
[[1]][[1]]
[1] "3300 Brodie Drive, South San Jose, San Jose"

[[1]][[2]]
[1] "4300 La Torre Avenue, South San Jose, San Jose"

[[1]][[3]]
[1] 132.59


[[2]]
[[2]][[1]]
[1] "3300 Brodie Drive, South San Jose, San Jose"

[[2]][[2]]
[1] "3700 McLaughlin Avenue, South San Jose, San Jose"

[[2]][[3]]
[1] 126.24


[[3]]
[[3]][[1]]
[1] "3300 Brodie Drive, South San Jose, San Jose"

[[3]][[2]]
[1] "400 Ginkgo Court, South San Jose, San Jose"

[[3]][[3]]
[1] 109.625


[[4]]
[[4]][[1]]
[1] "1700 Coyote Point Drive, Shoreview, San Mateo"

[[4]][[2]]
[1] "1800 Helene Court, East San Mateo, San Mateo"

[[4]][[3]]
[1] 80.985


[[5]]
[[5]][[1]]
[1] "1700 Coyote Point Drive, Shoreview, San Mateo"

[[5]][[2]]
[1] 

In [26]:
#question8
satisfy_tri<-0
notsatisfy_tri<-0
matrix<-as_adjacency_matrix(gcc,attr="mean_travel_time")
for(i in 1:1000){
    vertex<-sample(1:vcount(gcc),3,replace=FALSE)
    v1<-matrix[vertex[1],vertex[2]]
    v2<-matrix[vertex[2],vertex[3]]
    v3<-matrix[vertex[3],vertex[1]]
    if((v1+v2>v3)&&(v1+v3>v3)&&(v2+v3>v1)){
        satisfy_tri=satisfy_tri+1
    }else{
        notsatisfy_tri=notsatisfy_tri+1
    }
}
print(satisfy_tri)
print(notsatisfy_tri)
print(satisfy_tri/1000)

[1] 41
[1] 959
[1] 0.041


In [50]:
#question9
temptree<-make_empty_graph(n=vcount(mstree),directed=FALSE)
tempdfs<-dfs(mstree,root=1,neimode="all")
temporder<-as.matrix(tempdfs$order)
meanratio<-6000
cost<-0.0
tem<-c()
for(i in 1:(vcount(mstree)-1)){
    temptree<-add_edges(temptree,c(temporder[i],temporder[i+1]))
    tem2<-E(mstree)[get.edge.ids(mstree,c(temporder[i],temporder[i+1]))]$mean_travel_time
    if(length(tem2)>0){
        cost<-cost+tem2
        tem<-c(tem,tem2)
    }else{
        tem2<-meanratio*sqrt((V(mstree)[temporder[i]]-V(mstree)[temporder[i+1]])^2+(V(mstree)[temporder[i]]-V(mstree)[temporder[i+1]])^2)
        cost<-cost+tem2
        tem<-c(tem,tem2)
    }
}
#cost<-cost+meanratio*sqrt((V(mstree)[temporder[1]]-V(mstree)[temporder[1887]])^2+(V(mstree)[temporder[1]]-V(mstree)[temporder[1887]])^2)
print(cost)

+ 1/1880 vertex, named, from 0f0cae0:
[1] 1873


In [46]:
mstree2=as.undirected(as.directed(mstree,'mutual'),'each')
small<-sum(E(mstree)$mean_travel_time)
print(small)
big<-sum(E(mstree2)$mean_travel_time)
print(big)
temptree<-make_empty_graph(n=vcount(mstree),directed=FALSE)
tempdfs<-dfs(mstree,root=1,neimode="all")
temporder<-as.matrix(tempdfs$order)
print(length(temporder))
a= as_adjacency_matrix(gcc, type = "both", attr = "mean_travel_time")
a[temporder[1],temporder[1880]]

[1] 279408.2
[1] 558816.4
[1] 1880


[1] 257.83